# Occupation

### Introduction:

Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder\
                    .appName('data')\
                    .getOrCreate()

25/06/04 15:53:21 WARN Utils: Your hostname, kevin-llanos-Type1ProductConfigId resolves to a loopback address: 127.0.1.1; using 192.168.1.92 instead (on interface wlo1)
25/06/04 15:53:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/04 15:53:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user). 

### Step 3. Assign it to a variable called users.

In [2]:
import requests

url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user'

get_url = requests.get(url)


with open('data.csv', 'w', encoding='utf-8') as f:
    f.write(get_url.text)


occupation = spark.read.csv('data.csv', header=True, sep='|')

df_occu = pd.read_csv('data.csv', sep='|')

In [3]:
occupation.show()

+-------+---+------+-------------+--------+
|user_id|age|gender|   occupation|zip_code|
+-------+---+------+-------------+--------+
|      1| 24|     M|   technician|   85711|
|      2| 53|     F|        other|   94043|
|      3| 23|     M|       writer|   32067|
|      4| 24|     M|   technician|   43537|
|      5| 33|     F|        other|   15213|
|      6| 42|     M|    executive|   98101|
|      7| 57|     M|administrator|   91344|
|      8| 36|     M|administrator|   05201|
|      9| 29|     M|      student|   01002|
|     10| 53|     M|       lawyer|   90703|
|     11| 39|     F|        other|   30329|
|     12| 28|     F|        other|   06405|
|     13| 47|     M|     educator|   29206|
|     14| 45|     M|    scientist|   55106|
|     15| 49|     F|     educator|   97301|
|     16| 21|     M|entertainment|   10309|
|     17| 30|     M|   programmer|   06355|
|     18| 35|     F|        other|   37212|
|     19| 40|     M|    librarian|   02138|
|     20| 42|     F|    homemake

In [4]:
df_occu.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


### Step 4. Discover what is the mean age per occupation

In [6]:
from pyspark.sql.functions import col, mean, round

occupation.groupBy(col('occupation')).agg(round(mean(col('age')), 2).alias('promedio')).show()

+-------------+--------+
|   occupation|promedio|
+-------------+--------+
|    librarian|    40.0|
|      retired|   63.07|
|       lawyer|   36.75|
|         none|   26.56|
|       writer|   36.31|
|   programmer|   33.12|
|    marketing|   37.62|
|        other|   34.52|
|    executive|   38.72|
|    scientist|   35.55|
|      student|   22.08|
|     salesman|   35.67|
|       artist|   31.39|
|   technician|   33.15|
|administrator|   38.75|
|     engineer|   36.39|
|   healthcare|   41.56|
|     educator|   42.01|
|entertainment|   29.22|
|    homemaker|   32.57|
+-------------+--------+
only showing top 20 rows



In [8]:
df_occu.groupby('occupation', as_index=False)['age'].mean().round(2)

,occupation,age
0,administrator,38.75
1,artist,31.39
2,doctor,43.57
3,educator,42.01
4,engineer,36.39
5,entertainment,29.22
6,executive,38.72
7,healthcare,41.56
8,homemaker,32.57
9,lawyer,36.75


### Step 5. Discover the Male ratio per occupation and sort it from the most to the least

In [30]:
from pyspark.sql.functions import countDistinct, count

total_por_ocupacion = occupation.groupBy("occupation") \
    .agg(count("user_id").alias("total_ocupacion"))

In [31]:
hombres_por_ocupacion = occupation.filter(col("gender") == "M") \
    .groupBy("occupation") \
    .agg(count("user_id").alias("hombres"))


In [32]:
joined = hombres_por_ocupacion.join(total_por_ocupacion, on="occupation")


In [33]:
from pyspark.sql.functions import round

resultado = joined.withColumn(
    "male_ratio",
    round((col("hombres") / col("total_ocupacion")) * 100, 6)  # 6 decimales como en el ejemplo
)


In [36]:
resultado.orderBy(col('male_ratio'), ascending=False).show()

+-------------+-------+---------------+----------+
|   occupation|hombres|total_ocupacion|male_ratio|
+-------------+-------+---------------+----------+
|       doctor|      7|              7|     100.0|
|     engineer|     65|             67| 97.014925|
|   technician|     26|             27| 96.296296|
|      retired|     13|             14| 92.857143|
|   programmer|     60|             66| 90.909091|
|    executive|     29|             32|    90.625|
|    scientist|     28|             31| 90.322581|
|entertainment|     16|             18| 88.888889|
|       lawyer|     10|             12| 83.333333|
|     salesman|      9|             12|      75.0|
|     educator|     69|             95| 72.631579|
|      student|    136|            196| 69.387755|
|        other|     69|            105| 65.714286|
|    marketing|     16|             26| 61.538462|
|       writer|     26|             45| 57.777778|
|         none|      5|              9| 55.555556|
|administrator|     43|        

### Step 6. For each occupation, calculate the minimum and maximum ages

In [42]:
from pyspark.sql.functions import min, max

occupation.groupBy(col('occupation')).agg(min(col('age')).alias('minima_edad'),
                                          max(col('age')).alias('maxima_edad')).show()

+-------------+-----------+-----------+
|   occupation|minima_edad|maxima_edad|
+-------------+-----------+-----------+
|administrator|         21|         70|
|       artist|         19|         48|
|       doctor|         28|         64|
|     educator|         23|         63|
|     engineer|         22|         70|
|entertainment|         15|         50|
|    executive|         22|         69|
|   healthcare|         22|         62|
|    homemaker|         20|         50|
|       lawyer|         21|         53|
|    librarian|         23|         69|
|    marketing|         24|         55|
|         none|         11|         55|
|        other|         13|         64|
|   programmer|         20|         63|
|      retired|         51|         73|
|     salesman|         18|         66|
|    scientist|         23|         55|
|      student|         10|          7|
|   technician|         21|         55|
+-------------+-----------+-----------+
only showing top 20 rows



In [43]:
df_occu.groupby('occupation', as_index=False)['age'].agg(['min', 'max'])

,occupation,min,max
0,administrator,21,70
1,artist,19,48
2,doctor,28,64
3,educator,23,63
4,engineer,22,70
5,entertainment,15,50
6,executive,22,69
7,healthcare,22,62
8,homemaker,20,50
9,lawyer,21,53


### Step 7. For each combination of occupation and gender, calculate the mean age

In [45]:
from pyspark.sql.functions import avg
occupation.groupBy(col('occupation'), col('gender')).agg(avg(col('age')).alias('promedio')).show()

+-------------+------+------------------+
|   occupation|gender|          promedio|
+-------------+------+------------------+
|   technician|     M| 32.96153846153846|
|     educator|     F| 39.11538461538461|
|       lawyer|     F|              39.5|
|entertainment|     F|              31.0|
|       lawyer|     M|              36.2|
|      retired|     F|              70.0|
|      student|     F|             20.75|
|   healthcare|     F| 39.81818181818182|
|administrator|     M| 37.16279069767442|
|    marketing|     M|            37.875|
|     engineer|     F|              29.5|
|    homemaker|     F|34.166666666666664|
|       artist|     F|30.307692307692307|
|         none|     F|              36.5|
|       doctor|     M| 43.57142857142857|
|       writer|     F| 37.63157894736842|
|     educator|     M| 43.10144927536232|
|    scientist|     M| 36.32142857142857|
|   technician|     F|              38.0|
|       writer|     M| 35.34615384615385|
+-------------+------+------------

In [46]:
df_occu.groupby(['occupation', 'gender'])['age'].mean()

occupation     gender
administrator  F         40.638889
               M         37.162791
artist         F         30.307692
               M         32.333333
doctor         M         43.571429
educator       F         39.115385
               M         43.101449
engineer       F         29.500000
               M         36.600000
entertainment  F         31.000000
               M         29.000000
executive      F         44.000000
               M         38.172414
healthcare     F         39.818182
               M         45.400000
homemaker      F         34.166667
               M         23.000000
lawyer         F         39.500000
               M         36.200000
librarian      F         40.000000
               M         40.000000
marketing      F         37.200000
               M         37.875000
none           F         36.500000
               M         18.600000
other          F         35.472222
               M         34.028986
programmer     F         32.16666

### Step 8.  For each occupation present the percentage of women and men